 # <strong>Road networks and robustness to flooding on US Atlantic and Gulf barrier islands</strong>
 ## <strong>- Download elevations for the US Atlantic and Gulf barrier islands -</strong>
 ### The purpose of this notebook is to download CUDEM tiles from https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/. These tiles will be clipped to the extent of the study area and used to retrieve elevations for each network node.

In [1]:
### Packages

import os
import requests
import urllib
import ssl
from osgeo import gdal, ogr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import rasterio
import glob
%matplotlib inline

In [2]:
### Set working directory

path=''
os.chdir(path)

In [ ]:
### Download CUDEM tiles

# Create folders if they don't exist
outdir= '.\\Data\\CUDEM'
if not os.path.exists(outdir):
    os.makedirs(outdir)

outdir = '.\\Data\\CUDEM\\Tiles'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Retrive url list with cudem tiles and save it
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/urllist8483.txt'
r = requests.get(url, allow_redirects=True)
open('.\\Data\\CUDEM\\urllist8483.txt', 'wb').write(r.content)

## If urlopen error pops up [SSL: CERTIFICATE_VERIFY_FAILED] use this line of code but keep in mind that allowing the use of unverified ssl can introduce security risks if the 
## data source is not trustworthy
# ssl._create_default_https_context = ssl._create_unverified_context 

# Function to get the name of the file from the url
def get_filename(url):
    """
    Parses filename from given url
    """
    if url.find('/'):
        return url.rsplit('/', 1)[1]

#  Download tiles using url list
with open('.\\Data\\CUDEM\\urllist8483.txt') as f:
    flat_list=[word for line in f for word in line.split()]
url_list = flat_list

for url in url_list:
    # parse filename
    fname = get_filename(url)
    outfp = os.path.join(outdir, fname)
    # download the file if it does not exist already
    try:
        if not os.path.exists(outfp):
            print('Downloading', fname)
            r = urllib.request.urlretrieve(url, outfp)
    except:
        continue

In [ ]:
### Check if extent of raster tile and barrier island shp (200m buffer) overlap. If true, clip raster using polygon and save it 

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

# Create folder if it does no exist
outdir= '.\\Data\\CUDEM\\CUDEM_Clip'
if not os.path.exists(outdir):
    os.makedirs(outdir)

for filename1 in os.listdir('.\\Data\\CUDEM\\Tiles'):
    if filename1.endswith('.tif'):
        raster_dir=('.\\Data\\CUDEM\\Tiles\\{0}'.format(filename1))
        raster_name=filename1.replace ('.tif', '')
        raster = gdal.Open(raster_dir)
        
        # get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize
        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('.\\Data\\Barriers\\Buffers_200m'):
            if filename2.endswith('.shp'):
                vector_dir=('.\\Data\\Barriers\\Buffers_200m\\{0}'.format(filename2))
                vector_name=filename2.replace('.shp', '')
                vector = ogr.Open(vector_dir)
                
                # get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()
                
                # check if they intersect and if they do clip raster tile using polygon
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                        # output clipped raster
                        out_tif = os.path.join('.\\Data\\CUDEM\\CUDEM_Clip\\{0}_{1}.tif'.format(vector_name,raster_name))
                        # read the data
                        data = rasterio.open(raster_dir)
                        barrier = gpd.read_file(vector_dir)
                        # project the Polygon into same CRS as the grid
                        barrier = barrier.to_crs(crs=data.crs.data)
                        coords = getFeatures(barrier)
                        # clip raster with polygon
                        out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
                        # copy the metadata
                        out_meta = data.meta.copy()
                        out_meta.update({"driver": "GTiff",
                                        "height": out_img.shape[1],
                                        "width": out_img.shape[2],
                                        "transform": out_transform,
                                        "crs": "EPSG:4269"})
                        # write clipped raster to disk
                        with rasterio.open(out_tif, "w", **out_meta) as dest:
                            dest.write(out_img)       
                else:
                    continue

In [4]:
### With the clipped rasters, create CUDEM mosaic for each barrier 

# Create folder if it does no exist
outdir= '.\\Data\\CUDEM\\CUDEM_Mosaic'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Merge all clipped rasters that start with the same name (belong to the same barrier) in one mosaic
for vector in os.listdir('.\\Data\\Barriers\\Buffers_200m'):
            if vector.endswith('.shp'):
                vector_name= vector.replace('.shp', '')
                # list for the source files
                src_files_to_mosaic = []
                for raster in os.listdir('.\\Data\\CUDEM\\CUDEM_Clip'):
                            if raster.startswith(vector_name):
                                src = rasterio.open('.\\Data\\CUDEM\\CUDEM_Clip\{0}'.format(raster))
                                src_files_to_mosaic.append(src)
                                # merge function returns a single mosaic array and the transformation info
                                mosaic, out_trans = merge(src_files_to_mosaic)
                                # copy the metadata
                                out_meta = src.meta.copy()
                                # update the metadata
                                out_meta.update({"driver": "GTiff",
                                                 "height": mosaic.shape[1],
                                                 "width": mosaic.shape[2],
                                                 "transform": out_trans,
                                                 "crs": "EPSG:4269"})
                                # write the mosaic raster to disk
                                with rasterio.open(".\\Data\\CUDEM\\CUDEM_Mosaic\{0}.tif".format(vector_name), "w", **out_meta) as dest:
                                    dest.write(mosaic)
                                
                            else:
                                continue
                                           